In [ ]:
import numpy as np

In [ ]:
#TODO

# Abre o arquivo "grafo.txt" em modo de leitura
f = open("graph_4.txt", "r")

# Lê a primeira linha do arquivo para obter o número de vértices no grafo
vertices = int(f.readline())

# Cria a matriz de adjacencia a partir de grafos.txt
matriz_adj  = np.loadtxt("graph_4.txt", skiprows = 1, dtype = int)

# Função para criar uma lista de adjacências a partir da matriz de adjacência do grafo
def cria_list_adj(grafo):
    list_adj = dict()

    for i in range(vertices):
        # Encontra os índices onde a matriz de adjacência possui valor 1 para o vértice i
        adj_indices = np.where(grafo[i] == 1)[0]

        # Converte os índices em uma lista de vértices adjacentes ao vértice i
        adj_vertices = list(adj_indices)

        list_adj[i] = adj_vertices

    return list_adj

# Chama a função para criar a lista de adjacências
lista_adj = cria_list_adj(matriz_adj)

In [ ]:
lista_adj

{0: [1, 2, 7, 8, 9],
 1: [0, 2, 6, 9],
 2: [0, 1, 4, 5, 6, 7, 8],
 3: [5, 7, 9],
 4: [2, 5, 7, 8],
 5: [2, 3, 4, 6, 8],
 6: [1, 2, 5, 8, 9],
 7: [0, 2, 3, 4, 8, 9],
 8: [0, 2, 4, 5, 6, 7, 9],
 9: [0, 1, 3, 6, 7, 8]}

In [ ]:
matriz_adj

array([[0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
       [1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
       [1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 1],
       [1, 1, 0, 1, 0, 0, 1, 1, 1, 0]])

In [ ]:
class Grafo:
    def __init__(self, matriz_adj, lista_adj):
        self.matriz_adj = matriz_adj
        self.lista_adj = lista_adj
        self.vertices = self.inicializar_vertices()
        self.matriz_cor = self.inicializar_matriz_cor()
        self.matriz_distancia = self.inicializar_matriz_distancia()
        self.relogio = 0

    def inicializar_vertices(self):
      vertices = {}
      num_vertices = len(self.matriz_adj)

      for i in range(num_vertices):
          vertices[i] = {"visitado": False, "vizinhos": self.lista_adj[i], "pai": None, "irmãos": [],"primos": [], "tios": [], "retornos": [] ,"nivel": None, "exc": 0}

      return vertices

    def inicializar_matriz_cor(self):
        num_vertices = len(self.matriz_adj)
        matriz_cor = [['branco' for j in range(num_vertices)] for i in range(num_vertices)]
        return matriz_cor

    def inicializar_matriz_distancia(self):
        num_vertices = len(self.matriz_adj)
        matriz_distancia = [[0 for j in range(num_vertices)] for i in range(num_vertices)]
        return matriz_distancia

    def busca_profundidade(self, vertice):
        self.vertices[vertice]["visitado"] = True
        #self.relogio += 1
        #PE_vertice = self.relogio  # Entrada no vértice
        for vizinho in self.vertices[vertice]["vizinhos"]:
            if not self.vertices[vizinho]["visitado"]:
                #print(f"Visitar aresta {vertice} - {vizinho} (aresta azul da árvore de profundidade T)")
                self.vertices[vizinho]["pai"] = vertice
                self.busca_profundidade(vizinho)
            elif vizinho != self.vertices[vertice]["pai"]: # se o vizinho já foi visitado e não é o pai do vertice
                self.vertices[vizinho]["retornos"].append(vertice) # adiciona o vertice como retorno do vizinho
                #print(f"Visitar aresta {vertice} - {vizinho} (aresta vermelha de retorno)")

        #self.relogio += 1
        #PS_vertice = self.relogio  # Saída do vértice

        #print(f"Entrada ({PE_vertice}): PE({vertice}) -> N({vertice}) {self.vertices[vertice]['vizinhos']}")
        #print(f"Saída ({PS_vertice}): PS({vertice}) -> N({vertice}) {self.vertices[vertice]['vizinhos']}")

    def busca_largura(self, vertice):
      fila = [vertice]
      self.vertices[vertice]["visitado"] = True
      self.vertices[vertice]["nivel"] = 1
      self.relogio += 1

      while len(fila) != 0:
          vertice = fila[0]
          fila.pop(0)
          for vizinho in self.vertices[vertice]["vizinhos"]:
              if not self.vertices[vizinho]["visitado"]:
                  self.vertices[vizinho]["visitado"] = True
                  self.vertices[vizinho]["pai"] = vertice
                  self.vertices[vizinho]["nivel"] = self.vertices[vertice]["nivel"] + 1
                  self.relogio += 1
                  #print(f"Entrada ({self.relogio}): L({vertice}) ->N({vertice}) {self.lista_adj[vertice]}")
                  fila.append(vizinho)
              else:  # Vizinho já visitado
                  if self.vertices[vizinho]["nivel"] == self.vertices[vertice]["nivel"]:
                      if self.vertices[vizinho]["pai"] == self.vertices[vertice]["pai"]:
                          self.vertices[vizinho]["irmãos"].append(vertice)
                      else:
                          self.vertices[vizinho]["primos"].append(vertice)
                  elif self.vertices[vizinho]["nivel"] == self.vertices[vertice]["nivel"] + 1:
                      self.vertices[vizinho]["tios"].append(vertice)

    def preencher_matriz_cor(self):
        for vertice, conteudo in self.vertices.items():
          for tio in conteudo["tios"]: # para cada tio do meu vertice
              self.matriz_cor[vertice][tio] = 'verde'
              self.matriz_cor[tio][vertice] = 'verde'
          for irmao in conteudo["irmãos"]:
              self.matriz_cor[vertice][irmao] = 'vermelho'
              self.matriz_cor[irmao][vertice] = 'vermelho'
          for primo in conteudo["primos"]:
              self.matriz_cor[vertice][primo] = 'amarelo'
              self.matriz_cor[primo][vertice] = 'amarelo'
          for retorno in conteudo["retornos"]:
              self.matriz_cor[vertice][retorno] = 'vermelho'
              self.matriz_cor[retorno][vertice] = 'vermelho'
          pai = conteudo["pai"]
          if pai is not None:
              self.matriz_cor[vertice][pai] = 'azul'
              self.matriz_cor[pai][vertice] = 'azul'

    def distancia(self):
        for i in range(len(self.matriz_adj)):
          for j in range(len(self.matriz_adj)):
              self.matriz_distancia[i][j] = abs(self.vertices[i]["nivel"] - self.vertices[j]["nivel"])
              if self.matriz_distancia[i][j] > self.vertices[i]["exc"]:
                self.vertices[i]["exc"] = self.matriz_distancia[i][j] #comprimento do maior menor caminho
        #print("\nMatriz de distancias:\n")
        return self.matriz_distancia

    def criar_arquivo_gdf(self, output_filename):
        processed_edges = set()  # Conjunto de arestas processadas
        with open(output_filename, 'w') as file:  # Abre o arquivo para escrita
            file.write("nodedef>name VARCHAR,label VARCHAR\n")  # Define as colunas para nome e label
            for vertice, data in self.vertices.items():  # Para cada vértice e seus dados
                nome_vertice = f"s{vertice}"  # Nome do vértice
                label = str(vertice+1)  # Rótulo do vértice sendo o número do vértice
                file.write(f"{nome_vertice},{label}\n")  # Escreve a linha do vértice no arquivo

            file.write("edgedef>node1 VARCHAR,node2 VARCHAR,color VARCHAR\n")  # Escreve a segunda linha do arquivo
            for vertice in range(len(self.matriz_adj)):  # Para cada vértice
                for vizinho in self.vertices[vertice]["vizinhos"]:  # Para cada vizinho do vértice
                    classification = self.matriz_cor[vertice][vizinho]  # Classificação da aresta

                    if classification == 'vermelho':  # Se a classificação for vermelho
                        color = '255,0,0'
                    elif classification == 'azul':  # Se a classificação for azul
                        color = '0,0,255'
                    elif classification == 'amarelo':  # Se a classificação for amarelo
                        color = '255,255,0'
                    elif classification == 'verde':  # Se a classificação for verde
                        color = '0,255,0'

                    edge = (f"s{vertice}", f"s{vizinho}")  # Aresta
                    reverse_edge = (f"s{vizinho}", f"s{vertice}")  # Aresta reversa

                    if edge not in processed_edges and reverse_edge not in processed_edges: # evita arestas reversas pois não é direcionado
                        file.write(f"s{vertice},s{vizinho},'{color}'\n")  # Escreve a linha da aresta no arquivo
                        processed_edges.add(edge)  # Adiciona a aresta no conjunto de arestas processadas



In [ ]:
grafo = Grafo(matriz_adj, lista_adj)
grafo.busca_profundidade(0)
grafo.preencher_matriz_cor()
grafo.criar_arquivo_gdf("output.gdf")

In [ ]:
grafo = Grafo(matriz_adj, lista_adj)
grafo.busca_largura(0)
grafo.preencher_matriz_cor()
grafo.criar_arquivo_gdf("output.gdf")

In [ ]:
def diametro_raio(exc):
        diametro = 0
        raio = len(exc)
        for i in range(len(exc)):
          if exc[i] > diametro:
            diametro = exc[i]
          if exc[i] < raio:
            raio = exc[i]

        print("Diametro: ", diametro)
        print("Raio: ", raio)


dist = None
for i in range(len(matriz_adj)):
    grafo = Grafo(matriz_adj, lista_adj)
    grafo.busca_largura(i)
    distancia_atual = grafo.distancia()
    if dist is None:
        dist = distancia_atual
    else:
        dist = np.maximum(dist, distancia_atual) #maxima distancia entre os vertices

exc = []
for i in range(len(matriz_adj)):
  exc.append(np.max(dist[i]))

In [ ]:
exc

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [ ]:
diametro_raio(exc)
print(dist)

Diametro:  2
Raio:  2
[[0 1 1 2 2 2 2 1 1 1]
 [1 0 1 2 2 2 1 2 2 1]
 [1 1 0 2 1 1 1 1 1 2]
 [2 2 2 0 2 1 2 1 2 1]
 [2 2 1 2 0 1 2 1 1 2]
 [2 2 1 1 1 0 1 2 1 2]
 [2 1 1 2 2 1 0 2 1 1]
 [1 2 1 1 1 2 2 0 1 1]
 [1 2 1 2 1 1 1 1 0 1]
 [1 1 2 1 2 2 1 1 1 0]]


In [ ]:
#Somar todas as distâncias entre os vértices
soma_distancias = np.sum(dist)

# Dividir a soma pelo número total de pares de vértices
# Como a diagonal da matriz de distâncias é 0, tem que subtrair a quantidade de vértices (para desconsiderar a diagonal).
n = len(matriz_adj)
distancia_media = soma_distancias / (n * (n - 1)) # não tem /2 pois a matriz de distancias considera A pra B e B pra A quando somo tudo

print("Distância Média: ", distancia_media)

Distância Média:  1.4222222222222223
